In [28]:
# 1 : load image generation net pgan

import os
import random
import torch


from torch.autograd import Variable
from models.progressive_gan import ProgressiveGAN
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Resize
from torch.utils.serialization import load_lua
from torchvision.utils import make_grid,save_image
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Resize
from IPython.display import display, clear_output
from PIL import Image
to_pil = ToPILImage()
import PIL
import ipywidgets as widgets
import numpy as np
import json
import csv
from IPython.display import display, update_display, clear_output
from random import shuffle, randint
import getpass
Username = getpass.getuser()


#setting = "RTW"
#setting = "fashionGen"
setting = "celeba"
#setting = "udtd"
#sc = 5
#SZ= 4*pow(2,sc)
#sizez=512
sc = 6
SZ= 4*pow(2,sc)
sizez=512

if setting == "dtd20":
    dataset = 'PGAN_DTD20'
    sizez=303 #256+47
else:
    if setting == "celeba" or setting == "celebacartoon" or setting == "celebacartoon2" or setting == "celebabam" or setting == "celebasemi" or setting == "missceleba":
        dataset = 'celebaHQ16_december'
    else:
        if setting == "dtd20miss":
            dataset = 'PGAN_DTD20'
            sizez=303
        else:
            if setting == "udtd":
                dataset = 'pgan_dtd_uncond'
                sizez=303
            else: 
                if setting == "RTW":
                    dataset = 'yslcreativity4'
                    sizez=519
                else:
                    if setting == "udtd20bam" or setting == "udtd20clb" or setting == "yo":
                        dataset = 'PGAN_DTD20'
                        sizez=303
                    else:
                        if setting == "fashionGen":
                            dataset= 'fashionGenClothingCleared2'
                            sizez = 534 
                        else: 
                            assert False

iternb = '96000'
if dataset == 'PGAN_DTD20':
    pathModel = '/private/home/'+Username+'/Riviere2018Fashion/pytorch_GAN_zoo/PGAN_DTD20/default/default_s'+str(sc)+'_iter_48000.pt'
    pathConfig = '/private/home/'+Username+'/Riviere2018Fashion/pytorch_GAN_zoo/PGAN_DTD20/default/default_s'+str(sc)+'_iter_48000_tmp_config.json'
else:
    if dataset == 'pgan_dtd_uncond':
        pathModel = '/private/home/'+Username+'/Riviere2018Fashion/pytorch_GAN_zoo/pgan_dtd_uncond/default/default_s'+str(sc)+'_iter_48000.pt'
        pathConfig = '/private/home/'+Username+'/Riviere2018Fashion/pytorch_GAN_zoo/pgan_dtd_uncond/default/default_s'+str(sc)+'_iter_48000_tmp_config.json'
    else:
        if dataset == 'yslcreativity4':
            pathModel = '/private/home/'+Username+'/Riviere2018Fashion/cleangit/pytorch_GAN_zoo/'+dataset+'/'+dataset+'_s'+str(sc)+'_iter_'+iternb+'.pt'
            pathConfig = '/private/home/'+Username+'/Riviere2018Fashion/cleangit/pytorch_GAN_zoo/'+dataset+'/'+dataset+'_s'+str(sc)+'_iter_'+iternb+'_tmp_config.json'
        else:
            pathModel = '/private/home/'+Username+'/datasets/'+dataset+'/'+dataset+'_s'+str(sc)+'_iter_'+iternb+'.pt'
            pathConfig = '/private/home/'+Username+'/datasets/'+dataset+'/'+dataset+'_s'+str(sc)+'_iter_'+iternb+'_tmp_config.json'



with open(pathConfig, 'rb') as file:
    config = json.load(file)
    
pgan  = ProgressiveGAN(useGPU= True, storeAvG = True, **config)
print(pathModel)
pgan.load(pathModel)

# 2 Generate images from pgan ----------------------

nimages= 10
noiseData, noiseLabels = pgan.buildNoiseData(nimages)
img = pgan.test(noiseData, getAvG = True)

for i in range(0,nimages):
    
    img2 = img[i].view(3,SZ,SZ).add(1).div(2) 
    img2 = img2.data.cpu()
    img2 = np.clip(img2, 0, 1)
    out = to_pil(img2)
    out.save('/private/home/'+Username+'/HDGANSamples/random_gens/'+dataset+'_s'+str(sc)+'_rand_'+ str(i) +'.jpg')
    
#    new_im.paste(out, (x_offset,0))
#    x_offset += out.size[0]

#new_im.save('/private/home/'+Username+'/HDGANSamples/random_gens/'+dataset+'_s'+str(sc)+'_all_rand_' +'.jpg') 
#display(new_im)


#pathDB = '/private/home/'+Username+'/HDGANSamples/random_gens/'
#pathOut = "coin.json"
#nReviews = 4
#testInterface = Interface(pathDB)

/private/home/oteytaud/datasets/celebaHQ16_december/celebaHQ16_december_s6_iter_96000.pt
Average network found !


In [35]:

averaging_parents = False
uniform_mixing_of_mutation_rates = True

class Optim():
    
#    def __init__(self,d,llambda=15,mu=1, budget=100,mr=1):
    def __init__(self,d,llambda=23 + (4 if averaging_parents else 0), mu=5, budget=100, mr=None):
        # d = dimension of z
        # lambda = nb images that we generate to compare
        # mu = nb of selected images by the user
        
        self._d = d
        self._z = [[0.]*d]
        if not averaging_parents:
            for i in range(mu-1):
                self._z += [[0.]*d]
        self.llambda = llambda
        self.mu = mu
        self.budget = budget
        self.mr = mr if mr else 1. / d

    def ask(self):
        self._pop = self._z
        print("before offspring:", len(self._pop))
        
        mr = self.mr
        if uniform_mixing_of_mutation_rates:
            mr = max(random.uniform(0,1), 1. / self._d)
            
        for i in range(self.llambda):
            z = self._z[i % self.mu]
            while True: 
                zp = []
                for q in z: #j in range(d):
                    if random.uniform(0,1)<mr:#1./self._d:
                        zp+=[random.gauss(0,1)]
                    else:
                        zp+=[q]
                        
                if self._z != zp:
                    break
            self._pop += [zp] 
        self.mr = 1. / 5. #self._d
        print("after offspring:", len(self._pop))
        assert(len(self._pop)==((self.mu if not averaging_parents else 1) + self.llambda))
        self.budget -=1
        print(self.budget)
        # generate images
        
        img = pgan.test(torch.FloatTensor(self._pop))
        img_list = []
        for i in range(0,self.llambda+1):
    
            img2 = img[i].view(3,SZ,SZ).add(1).div(2) 
            img2 = img2.data.cpu()
            img2 = np.clip(img2, 0, 1)
            out = to_pil(img2)
            #display(out)
            img_list+=[out]
        
        
        return img_list,self.mu,"click on "+str(self.mu)+" image(s)"
                
    def tell(self, indices):
        self._z = [] #[0.]*d
        assert(len(indices)==self.mu)
        if averaging_parents:
            for i in range(self._d):
                q=0
                for j in range(self.mu):
                    q+=self._pop[indices[j]][i]
                self._z+=[q/self.mu]
            self._z = [self._z]
        else:
            for j in range(self.mu):
                self._z += [self._pop[indices[j]]]

    def terminate(self):
        return (self.budget<=0)
    
# Interface -----------------------------------------------

import ipywidgets as widgets
from io import BytesIO
from IPython.display import display, update_display, clear_output

def getImages(pathDir, extension=".jpg"):
    return [x for x in os.listdir(pathDir) if os.path.splitext(x)[1] == extension]

class mockInterface:
    
    def __init__(self, pathDB, k):
        self.imagePath = os.path.join(pathDB, getImages(pathDB)[0])
        self.k = k
        self.iter = 0
        self.index  = []
        
    def ask(self):
        extention = os.path.splitext(self.imagePath)[1][1:]
        image = PIL.Image.open(self.imagePath, mode='r')
        return [image]*10, self.k, f"Click on {self.k} images"
    
    def tell(self, indexList):
        self.iter+=1
        self.index.append(indexList)
        
    def terminate(self):
        return self.iter > 2

class Interface:
    
    def __init__(self, optimizer):
        self.optimizer = optimizer
        self.outChoices = []
        self.toChoose = -1
        
    def onClick(self, b):
        self.outChoices.append(b.index)
        print(b.index)
        #b.disabled=True
        
        if len(self.outChoices) == self.toChoose:
            clear_output()
            self.optimizer.tell(self.outChoices)
            self.continueReview()
        
    def showImages(self, imageList):
        
        toShow1 = []
        toShow2 = []
        toShow3 =[]
        toShow4 = []
        self.buttons = []
        for i, image in enumerate(imageList):
            b = BytesIO()
            image.save(b, format='png')
            bytestream = b.getvalue()
            data = widgets.Image(value=bytestream, format='png', embed=True)
            self.buttons.append(widgets.Button(description=str(i),disabled=False))
            if i % 4 == 0:
                toShow1.append(widgets.VBox([data, self.buttons[-1]]))
            if i % 4 == 1:
                toShow2.append(widgets.VBox([data, self.buttons[-1]]))
            if i % 4 == 2:
                toShow3.append(widgets.VBox([data, self.buttons[-1]]))
            if i % 4 == 3:
                toShow4.append(widgets.VBox([data, self.buttons[-1]]))
            self.buttons[-1].index = i
            self.buttons[-1].on_click(self.onClick)
                
        display(widgets.VBox([
            widgets.HBox(toShow1),
            widgets.HBox(toShow2),
            widgets.HBox(toShow3),
            widgets.HBox(toShow4),
        ]))
        
    def continueReview(self):
        
        if self.optimizer.terminate():
            return
        
        imageList, k, text = self.optimizer.ask()
        self.toChoose = k
        self.outChoices = []
        print(text)
        self.showImages(imageList)
        
    def startShow(self):
        self.continueReview()


optim = Optim(sizez)
inter = Interface(optim)   


In [36]:
inter.startShow()

before offspring: 5
after offspring: 28
98
click on 5 image(s)


A Jupyter Widget